### Question 1 Part A
Topics 2-3, (Size, Equality, Norms and Sensitivity) and (Matrix Structure and Factorisations)

This question asks to generate and compute several different statistics from the matrix

In [11]:
import numpy as np
from numpy import pi, cos
from numpy.linalg import cond, det
from scipy.linalg import norm, qr

# (a).  Generating a matrix and sum
n = 12
def f(i, j):
    if i == 0:
        if j == 0:
            return 1
        else:
            return 0
    return 1/((2*i + 1)**j)
A = np.zeros((n,n))
for i in range(0, n):
    for j in range(0, n):
        A[i,j] = f(i, j)

sumA = sum(sum(A))
print(f"sumA: {sumA}")

condA = cond(A)
print(f"condA: {condA}")

comment1='yes'

detA = det(A)
print(f"detA: {detA}")

comment2='no'

q,r = qr(A)
diag5Q = np.diag(q)[:5]
print(f"diag5Q: {diag5Q}")

sumA: 13.509935844715358
condA: 2.0778039513457235e+17
detA: -7.584782657868772e-83
diag5Q: [-0.28867513 -0.77990597 -0.35925163  0.14627472 -0.40354541]


In [15]:
def f(x):
    return np.sin(10*np.pi*x)/(100*np.exp(x))
x = np.array([0, 1/9, 1/7, 1/5, 1/3])
A = np.vander(x, N=5, increasing=True)

veca = np.linalg.solve(A, f(x))

print(veca)

[   0.            0.86037863  -15.50499537   82.13272647 -130.58607689]


### Question 2
Newton's Method

In [4]:
from numpy import zeros, exp

x0 = 3

def f(x):
    return (x-1)**4 + exp(x)

def g(x):
    return f(x) - 2

def g_prime(x):
    return 4*(x-1)**3 + exp(x)

def newton_step(x):
    x2 = x - (g(x)/g_prime(x))
    return x2

x1 = newton_step(x0)
x2 = newton_step(x1)
x3 = newton_step(x2)

print(f"x1: {x1}")
print(f"x2: {x2}")
print(f"x3: {x3}")

tol = 1e-8
cur_x = x0
while True:
    xe = newton_step(cur_x)
    if (abs(xe - cur_x) < tol):
        break
    cur_x = xe
print(f"xe: {xe}")

x1: 2.3455853786540635
x2: 1.7650623277302933
x3: 1.2168575333869849
xe: 0.6884238065328873


### Question 5
Boundary-Value Problems

In [35]:
from numpy.linalg import solve, norm
from numpy import diag, exp, ones, zeros, linspace, inf
import numpy as np

def perform_system(N = 5):
    h = 1/N

    x_all = [n*h for n in range(0, N+1)]
    x = x_all[1:-1]

    def p(x):
        return exp(-3*x)

    def q(x):
        return 1

    A = diag([-2] * (N-1)) + diag([1] * (N-2), 1) + diag([1] * (N-2), -1)
    B_init = diag([-1] * (N-2), -1) + diag([1] * (N-2), 1)
    B = diag([p(xi) for xi in x]) @ B_init
    C = diag([q(xi) for xi in x])
    R = np.zeros(N-1)
    R[-1] = 1/h**2 - 3/(2*h) * exp(-3 * (N-1) * h)

    LHS = (-1 / h**2) * A + (3 / (2 * h)) * B + 9*C
    RHS = R
    Uapprox = solve(LHS, RHS)
    return Uapprox

perform_system(5)


array([0.04368579, 0.12341851, 0.2678357 , 0.52877524])

In [36]:
def u(x):
    return (1-exp(3*x))/(1-exp(3))

U1 = perform_system(40)
U2 = perform_system(80)

u1_true = np.array([u(x) for x in linspace(0, 1, 40)])[:-1]
u2_true = np.array([u(x) for x in linspace(0, 1, 80)])[:-1]

print(U1.shape)
print(u1_true.shape)

print(norm(U1 - u1_true, inf))
print(norm(U2 - u2_true, inf))

(39,)
(39,)
0.009859587751005316
0.004883714433524977


### Question 4
Cubic Spline Interpolation, LSQ Fitting, DFT

In [97]:
from numpy import ones, linspace, loadtxt, pi, exp, cos, sin
from scipy.interpolate import CubicSpline
from scipy.fft import fft
from scipy.linalg import lstsq

tdata = np.arange(1, 11)
ydata = np.array([77.64, 94.53, 68.17, 39.68, 56.99, 65.23, 50.8, 43.29, 48.66, 93.17])

spl = CubicSpline(tdata, ydata, bc_type='not-a-knot')

teval = np.arange(1, 10) + 0.3
yspteval = spl(teval)
print(yspteval)

A = np.column_stack((np.array([1] * len(tdata)), tdata, exp(tdata), cos(pi*tdata/2)))
coeffs, *_ = lstsq(A, ydata)
Aeval = np.column_stack((np.array([1] * len(teval)), teval, exp(teval), cos(pi*teval/2)))
yLSteval = Aeval @ coeffs
print(yLSteval)

print("------------------")

m = 10

w = fft(ydata)
a0 = w[0].real / m

a = zeros(5)
for k in range(0, 4):
    a[k] = (2 / m) * w[k+1].real
a[4]=(w[5].real/m)
b = [3.9119, 1.7992, -0.4331, -4.9655, 0]
ydfteval = [a0 + np.sum([a[i-1] * cos(2*pi*i*t/m) + b[i-1] * sin(2*pi*i*t/m) for i in range(1,6)]) for t in teval]
print(ydfteval)


[89.11551718 89.7353398  56.78286361 41.37721576 62.41078334 62.02855087
 47.29204316 42.80659648 56.36219093]
[83.44709974 85.27932684 59.24535957 47.4801628  63.76370156 66.27466822
 42.08570637 35.33575429 65.25214159]
------------------
[94.80744577729573, 53.98808519494918, 42.043406834250746, 63.249650061961134, 60.52863068999574, 49.50790369246462, 39.145054674855146, 63.88337010733187, 92.44200996716202]


### Question 3
Numerical Integration

In [126]:
N=9
h = (1-0)/N

print(N)
print(h)

x = linspace(0, 1, N+1)
print(x)

w = np.empty(N+1)
w[0] = w[-1] = 1
w[1:-1] = 2
print(w)

def f(x):
    return (2-2*x**2)**(1.5)*np.sqrt(x)

f = [f(xi) for xi in x]
print(f)

trapf = ((0.5 * w).T @ f)*h
print(trapf)

from scipy.special import gamma
print(16/(15*pi**0.5)*(gamma(0.75))**2)
errtrap = np.abs(16/(15*pi**0.5)*(gamma(0.75))**2 - trapf)
print(errtrap)

9
0.1111111111111111
[0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]
[1. 2. 2. 2. 2. 2. 2. 2. 2. 1.]
[0.0, 0.9254036134008974, 1.2357974537342358, 1.368533971412446, 1.3554897600629898, 1.2118901430400677, 0.9562921842487448, 0.6193985693941798, 0.2563979721463201, 0.0]
0.8810226297155422
0.9036939571135776
0.022671327398035346


In [ ]:
N, h = 9, 1/9
x = y = linspace(0, 1, N+1)

X,Y = np.meshgrid(x,y,indexing='ij')

F = sin(X * Y) * np.sqrt(X + 2 * Y)
trap2dF = (h**2 / 4) * w @ F @ w.T

print(trap2dF)

0.33498855671317784
